**Init**

In [1]:
!pip install "dask[complete]"
!pip install pyarrow pandas
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import necessary Libraries**

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
import datetime
from datetime import timedelta
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import dask.dataframe as dd
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

**Read Data**

In [4]:
pd.set_option('display.max_colwidth', None)


In [5]:
# Read the parquet file using Dask
ddf = dd.read_parquet('/content/drive/MyDrive/merged_data.parquet')
df = ddf.compute()
df.head()

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03


**Preprocess Data**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12332488 entries, 0 to 798151
Data columns (total 26 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   CUST_CUSTNO            string        
 1   ACC_BUSINESSTYPE       string        
 2   ACC_ACCNO              string        
 3   ACC_BUSINESSNO         string        
 4   ACC_CURRENCYISO        string        
 5   BUSINESSNO_TRANS       string        
 6   FK_CURRENCY            string        
 7   AMOUNT                 float64       
 8   AMOUNTORIG             float64       
 9   BRANCH_OFFICE          string        
 10  CONTRA_ACCNO           string        
 11  CONTRA_ZIP             string        
 12  CONTRA_NAME            string        
 13  CSHYN                  string        
 14  REASON1                string        
 15  REASON2                string        
 16  REASON4                string        
 17  TR_FLAG_01             string        
 18  ANALYTICAL_TRANS_CODE  stri

In [7]:
valeurs_manquantes = df.isnull().sum()
valeurs_manquantes

CUST_CUSTNO                     0
ACC_BUSINESSTYPE           176301
ACC_ACCNO                       0
ACC_BUSINESSNO                  0
ACC_CURRENCYISO            176301
BUSINESSNO_TRANS                0
FK_CURRENCY                     0
AMOUNT                          0
AMOUNTORIG                      0
BRANCH_OFFICE                   0
CONTRA_ACCNO                    0
CONTRA_ZIP               11298901
CONTRA_NAME              11298625
CSHYN                           0
REASON1                     73553
REASON2                   3371014
REASON4                  10155319
TR_FLAG_01                      0
ANALYTICAL_TRANS_CODE           0
TR_SP_01                        0
TR_SP_02                        0
TR_SP_03                        0
TR_SP_04                        0
TR_SP_05                 11298625
TR_SP_10                 12322292
VALUEDATE                       0
dtype: int64

In [8]:
for column in df.columns:
    if pd.api.types.is_string_dtype(df[column]):
        df[column] = df[column].fillna('')
    elif pd.api.types.is_numeric_dtype(df[column]):
        df[column] = df[column].fillna(0)

In [9]:
valeurs_manquantes = df.isnull().sum()
valeurs_manquantes

CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO           0
ACC_CURRENCYISO          0
BUSINESSNO_TRANS         0
FK_CURRENCY              0
AMOUNT                   0
AMOUNTORIG               0
BRANCH_OFFICE            0
CONTRA_ACCNO             0
CONTRA_ZIP               0
CONTRA_NAME              0
CSHYN                    0
REASON1                  0
REASON2                  0
REASON4                  0
TR_FLAG_01               0
ANALYTICAL_TRANS_CODE    0
TR_SP_01                 0
TR_SP_02                 0
TR_SP_03                 0
TR_SP_04                 0
TR_SP_05                 0
TR_SP_10                 0
VALUEDATE                0
dtype: int64

In [10]:
label_encoders = {}
list_columns = ['ACC_BUSINESSTYPE', 'ACC_CURRENCYISO', 'TR_SP_01', 'TR_SP_02', 'TR_SP_03', 'TR_SP_04', 'TR_SP_05', 'TR_SP_10']

In [11]:
for column in list_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [12]:

string_columns = [col for col in df.columns if pd.api.types.is_string_dtype(df[col])]
string_columns_without_list = [col for col in string_columns if col not in list_columns]
string_columns_without_list

['CUST_CUSTNO',
 'ACC_ACCNO',
 'ACC_BUSINESSNO',
 'BUSINESSNO_TRANS',
 'FK_CURRENCY',
 'BRANCH_OFFICE',
 'CONTRA_ACCNO',
 'CONTRA_ZIP',
 'CONTRA_NAME',
 'CSHYN',
 'REASON1',
 'REASON2',
 'REASON4',
 'TR_FLAG_01',
 'ANALYTICAL_TRANS_CODE']

In [13]:
label_encoders = {}

for column in string_columns_without_list:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [14]:
df.head()

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,41906,2,32883,390,11,4250603,15,-650.0,-650.0,0,...,0,0,14,9840297,9,65,11,0,0,2023-12-03
1,41906,2,32883,390,11,3737340,15,-34.0,-34.0,0,...,0,0,14,7763744,9,65,11,0,0,2023-12-03
2,51482,2,585601,390,11,3999917,15,120000.0,120000.0,0,...,0,1,13,1481031,10,65,11,0,0,2023-12-03
3,155161,2,102456,390,11,3947486,15,-62.0,-62.0,0,...,0,0,14,10829756,9,65,11,0,0,2023-12-03
4,169438,2,434538,390,11,3789494,15,-1500.0,-1500.0,0,...,0,0,14,3667018,9,65,11,0,0,2023-12-03


**Apply FP Growth Mining**

In [15]:
# Apply FP-Growth algorithm
frequent_itemsets= fpgrowth(df.applymap(lambda x: 1 if x else 0), min_support=0.9, use_colnames=True)

In [16]:
frequent_itemsets

,support,itemsets
0,1.000000,(VALUEDATE)
1,1.000000,(TR_SP_01)
2,1.000000,(BUSINESSNO_TRANS)
3,1.000000,(ACC_BUSINESSNO)
4,0.999999,(ACC_ACCNO)
...,...,...
78842,0.903701,"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)"
78843,0.903701,"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)"
78844,0.903701,"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)"
78845,0.903701,"(ACC_ACCNO, AMOUNTORIG, TR_SP_04, CUST_CUSTNO, VALUEDATE, CONTRA_ACCNO, TR_SP_03, ACC_BUSINESSNO, TR_SP_01, ANALYTICAL_TRANS_CODE, AMOUNT, FK_CURRENCY)"


**Generate association rules**

In [17]:
# lower the value of confidence

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(VALUEDATE),(TR_SP_01),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00
1,(TR_SP_01),(VALUEDATE),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,inf,0.000000e+00
2,(BUSINESSNO_TRANS),(VALUEDATE),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,inf,0.000000e+00
3,(VALUEDATE),(BUSINESSNO_TRANS),1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00
4,(BUSINESSNO_TRANS),(TR_SP_01),1.000000,1.000000,1.000000,1.000000,1.000000,-6.439294e-15,1.000000,-7.941252e-08
...,...,...,...,...,...,...,...,...,...,...
47731039,(TR_SP_03),"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.999992,0.903708,0.903701,0.903708,0.999999,-7.417558e-07,0.999992,-9.629226e-02
47731040,(TR_SP_01),"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",1.000000,0.903701,0.903701,0.903701,1.000000,-7.808574e-09,1.000000,-9.629915e-02
47731041,(ANALYTICAL_TRANS_CODE),"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, TR_SP_01, ACC_BUSINESSNO, FK_CURRENCY)",0.999448,0.904253,0.903701,0.904200,0.999942,-5.285607e-05,0.999448,-9.580007e-02
47731042,(ACC_BUSINESSNO),"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, FK_CURRENCY)",1.000000,0.903701,0.903701,0.903701,1.000000,-4.685124e-08,1.000000,-9.629878e-02


**Filter the best rules by metrics**

In [18]:
# prompt: make a code to filter the best rules  by zhangs metrics >=1 , convisction <=1.303028

def filter_rules(rules):
  filtered_rules = rules[(rules['zhangs_metric'] >= 1) & (rules['conviction'] <= 1.303028)]
  return filtered_rules

best_rules = filter_rules(rules)
best_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
11453188,(ACC_CURRENCYISO),"(TR_SP_02, REASON1, AMOUNTORIG, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,1.341940e-02,1.285523,1.0
11453189,(ACC_BUSINESSTYPE),"(ACC_CURRENCYISO, TR_SP_02, REASON1, AMOUNTORIG)",0.985704,0.938705,0.938705,0.952319,1.014503,1.341940e-02,1.285523,1.0
11453511,"(ACC_CURRENCYISO, VALUEDATE)","(TR_SP_02, REASON1, AMOUNTORIG, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,1.341940e-02,1.285523,1.0
11453512,"(VALUEDATE, ACC_BUSINESSTYPE)","(ACC_CURRENCYISO, TR_SP_02, REASON1, AMOUNTORIG)",0.985704,0.938705,0.938705,0.952319,1.014503,1.341940e-02,1.285523,1.0
11453520,(ACC_CURRENCYISO),"(TR_SP_02, VALUEDATE, AMOUNTORIG, REASON1, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,1.341940e-02,1.285523,1.0
...,...,...,...,...,...,...,...,...,...,...
47730979,"(AMOUNTORIG, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,2.185342e-02,1.303027,1.0
47731010,"(AMOUNT, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,2.185342e-02,1.303027,1.0
47731032,(BUSINESSNO_TRANS),"(ACC_ACCNO, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",1.000000,0.903701,0.903701,0.903701,1.000000,7.327806e-08,1.000001,1.0
47731033,(AMOUNTORIG),"(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,2.185342e-02,1.303027,1.0


In [19]:
#  display the top rules

best_rules.sort_values(by='confidence', ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
34421283,(ACC_BUSINESSTYPE),"(TR_SP_02, AMOUNTORIG, REASON1, AMOUNT, ACC_CURRENCYISO)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34421332,"(ACC_CURRENCYISO, VALUEDATE)","(AMOUNT, TR_SP_02, REASON1, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34421282,(ACC_CURRENCYISO),"(TR_SP_02, AMOUNTORIG, REASON1, AMOUNT, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
11453189,(ACC_BUSINESSTYPE),"(ACC_CURRENCYISO, TR_SP_02, REASON1, AMOUNTORIG)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34421470,(ACC_CURRENCYISO),"(TR_SP_02, AMOUNTORIG, VALUEDATE, REASON1, AMOUNT, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34421458,"(VALUEDATE, ACC_BUSINESSTYPE)","(TR_SP_02, AMOUNTORIG, REASON1, AMOUNT, ACC_CURRENCYISO)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34421457,"(ACC_CURRENCYISO, VALUEDATE)","(TR_SP_02, AMOUNTORIG, REASON1, AMOUNT, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34421345,(ACC_BUSINESSTYPE),"(TR_SP_02, VALUEDATE, REASON1, AMOUNT, ACC_CURRENCYISO)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34421471,(ACC_BUSINESSTYPE),"(TR_SP_02, AMOUNTORIG, VALUEDATE, REASON1, AMOUNT, ACC_CURRENCYISO)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0
34420888,(ACC_CURRENCYISO),"(AMOUNT, TR_SP_02, REASON1, ACC_BUSINESSTYPE)",0.985704,0.938705,0.938705,0.952319,1.014503,0.013419,1.285523,1.0


In [20]:
# prompt: make a code to filter the best rules  by zhangs metrics >=1 , convisction <=1.303028

def filter_rules(rules):
  filtered_rules = rules[(rules['zhangs_metric'] >= 1) & (rules['conviction'] >= 1.303027	)]
  return filtered_rules

best_rules2 = filter_rules(rules)
best_rules2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
523250,(ACC_CURRENCYISO),(ACC_BUSINESSTYPE),0.985704,0.985704,0.985704,1.000000,1.014503,0.014091,inf,1.0
523251,(ACC_BUSINESSTYPE),(ACC_CURRENCYISO),0.985704,0.985704,0.985704,1.000000,1.014503,0.014091,inf,1.0
523274,"(VALUEDATE, ACC_CURRENCYISO)",(ACC_BUSINESSTYPE),0.985704,0.985704,0.985704,1.000000,1.014503,0.014091,inf,1.0
523276,"(VALUEDATE, ACC_BUSINESSTYPE)",(ACC_CURRENCYISO),0.985704,0.985704,0.985704,1.000000,1.014503,0.014091,inf,1.0
523277,(ACC_CURRENCYISO),"(VALUEDATE, ACC_BUSINESSTYPE)",0.985704,0.985704,0.985704,1.000000,1.014503,0.014091,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
47730970,"(AMOUNT, BUSINESSNO_TRANS)","(ACC_ACCNO, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303028,1.0
47730979,"(AMOUNTORIG, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47731010,"(AMOUNT, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47731033,(AMOUNTORIG),"(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0


In [21]:
# prompt: display the rules starting by conviction 1.303027

best_rules2.sort_values(by='conviction', ascending=True).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
47731038,(AMOUNT),"(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47731033,(AMOUNTORIG),"(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47722843,(AMOUNTORIG),"(ACC_ACCNO, TR_SP_04, CUST_CUSTNO, VALUEDATE, CONTRA_ACCNO, TR_SP_03, ACC_BUSINESSNO, TR_SP_01, ANALYTICAL_TRANS_CODE, AMOUNT, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47722819,"(AMOUNT, VALUEDATE)","(ACC_ACCNO, AMOUNTORIG, TR_SP_04, CUST_CUSTNO, CONTRA_ACCNO, TR_SP_03, ACC_BUSINESSNO, TR_SP_01, ANALYTICAL_TRANS_CODE, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47722852,(AMOUNT),"(ACC_ACCNO, AMOUNTORIG, TR_SP_04, CUST_CUSTNO, VALUEDATE, CONTRA_ACCNO, TR_SP_03, ACC_BUSINESSNO, TR_SP_01, ANALYTICAL_TRANS_CODE, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47730979,"(AMOUNTORIG, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47731010,"(AMOUNT, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47718750,(AMOUNTORIG),"(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47722789,"(AMOUNTORIG, VALUEDATE)","(ACC_ACCNO, TR_SP_04, CUST_CUSTNO, CONTRA_ACCNO, TR_SP_03, ACC_BUSINESSNO, TR_SP_01, ANALYTICAL_TRANS_CODE, AMOUNT, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0
47706476,(AMOUNT),"(ACC_ACCNO, AMOUNTORIG, TR_SP_04, CUST_CUSTNO, CONTRA_ACCNO, TR_SP_03, ACC_BUSINESSNO, TR_SP_01, ANALYTICAL_TRANS_CODE, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0


In [27]:
# prompt: filter rules with required_fields = {'BUSINESSNO_TRANS', 'VALUEDATE', 'CUST_CUSTNO'}

required_fields = {'BUSINESSNO_TRANS', 'VALUEDATE', 'CUST_CUSTNO'}
filtered_rules = best_rules2[best_rules2['antecedents'].apply(lambda x: any(field in x for field in required_fields))]
filtered_rules = filtered_rules[filtered_rules['consequents'].apply(lambda x: any(field in x for field in required_fields))]
filtered_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
523852,"(ACC_CURRENCYISO, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, ACC_BUSINESSTYPE)",0.985704,0.985704,0.985704,0.999999,1.014503,0.014091,19309.948109,1.0
523853,"(VALUEDATE, ACC_BUSINESSTYPE)","(ACC_ACCNO, BUSINESSNO_TRANS, ACC_CURRENCYISO)",0.985704,0.985704,0.985704,0.999999,1.014503,0.014091,19309.948109,1.0
523971,"(ACC_CURRENCYISO, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, TR_SP_01, ACC_BUSINESSTYPE)",0.985704,0.985704,0.985704,0.999999,1.014503,0.014091,17379.051868,1.0
523972,"(VALUEDATE, ACC_BUSINESSTYPE)","(ACC_ACCNO, BUSINESSNO_TRANS, ACC_CURRENCYISO, TR_SP_01)",0.985704,0.985704,0.985704,0.999999,1.014503,0.014091,17379.051868,1.0
524179,"(ACC_CURRENCYISO, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, ACC_BUSINESSNO, ACC_BUSINESSTYPE)",0.985704,0.985703,0.985703,0.999999,1.014503,0.014091,11586.363147,1.0
...,...,...,...,...,...,...,...,...,...,...
47730767,"(AMOUNT, BUSINESSNO_TRANS, VALUEDATE)","(ACC_ACCNO, AMOUNTORIG, TR_SP_04, CUST_CUSTNO, CONTRA_ACCNO, TR_SP_01, ANALYTICAL_TRANS_CODE, FK_CURRENCY, ACC_BUSINESSNO, TR_SP_03)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303028,1.0
47730965,"(BUSINESSNO_TRANS, AMOUNTORIG)","(ACC_ACCNO, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303028,1.0
47730970,"(AMOUNT, BUSINESSNO_TRANS)","(ACC_ACCNO, AMOUNTORIG, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, VALUEDATE, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303028,1.0
47730979,"(AMOUNTORIG, VALUEDATE)","(ACC_ACCNO, BUSINESSNO_TRANS, CUST_CUSTNO, TR_SP_04, CONTRA_ACCNO, AMOUNT, TR_SP_03, TR_SP_01, ANALYTICAL_TRANS_CODE, ACC_BUSINESSNO, FK_CURRENCY)",0.975818,0.903701,0.903701,0.926096,1.024781,0.021853,1.303027,1.0


In [28]:
# prompt: from the filtered rules display the top 10

filtered_rules.sort_values(by='confidence', ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
14283429,"(AMOUNTORIG, VALUEDATE)","(AMOUNT, BUSINESSNO_TRANS)",0.975818,0.975818,0.975818,1.0,1.024781,0.023597,291015.231535,1.0
14283426,"(AMOUNT, VALUEDATE)","(BUSINESSNO_TRANS, AMOUNTORIG)",0.975818,0.975818,0.975818,1.0,1.024781,0.023597,291015.231535,1.0
17456256,"(AMOUNT, VALUEDATE, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, AMOUNTORIG)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,456251.668778,1.0
17456255,"(AMOUNT, ACC_CURRENCYISO, VALUEDATE)","(BUSINESSNO_TRANS, AMOUNTORIG, ACC_BUSINESSTYPE)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,456251.668778,1.0
17456251,"(AMOUNTORIG, VALUEDATE, ACC_BUSINESSTYPE)","(AMOUNT, BUSINESSNO_TRANS, ACC_CURRENCYISO)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,456251.668778,1.0
17456250,"(ACC_CURRENCYISO, AMOUNTORIG, VALUEDATE)","(AMOUNT, BUSINESSNO_TRANS, ACC_BUSINESSTYPE)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,456251.668778,1.0
17456546,"(ACC_CURRENCYISO, AMOUNTORIG, VALUEDATE)","(AMOUNT, BUSINESSNO_TRANS, TR_SP_01, ACC_BUSINESSTYPE)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,228126.315301,1.0
17456558,"(AMOUNT, VALUEDATE, ACC_BUSINESSTYPE)","(BUSINESSNO_TRANS, ACC_CURRENCYISO, AMOUNTORIG, TR_SP_01)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,228126.315301,1.0
17456557,"(AMOUNT, ACC_CURRENCYISO, VALUEDATE)","(BUSINESSNO_TRANS, AMOUNTORIG, TR_SP_01, ACC_BUSINESSTYPE)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,228126.315301,1.0
17456547,"(AMOUNTORIG, VALUEDATE, ACC_BUSINESSTYPE)","(AMOUNT, BUSINESSNO_TRANS, ACC_CURRENCYISO, TR_SP_01)",0.961524,0.961524,0.961524,1.0,1.040016,0.036996,228126.315301,1.0
